# Crear tablas de dimensiones y hechos en el Warehouse

## DIM_DATE

In [1]:
from pyspark.sql import functions as F
from datetime import datetime, timedelta
from delta.tables import DeltaTable

print("="*70)
print("Generando dim_date")
print("="*70)

# 1. Obtener los límites de fechas de las tablas Silver
df_articles = spark.read.table("gold_articles")
df_blogs = spark.read.table("gold_blogs")
df_reports = spark.read.table("gold_reports")

dates_union = df_articles.select("published_at") \
    .union(df_blogs.select("published_at")) \
    .union(df_reports.select("published_at"))

# Extraer el valor mínimo y máximo
stats = dates_union.select(
    F.min("published_at").cast("date"), 
    F.max("published_at").cast("date")
).collect()

start_date = stats[0][0]
end_date = stats[0][1]

print(f"Rango detectado: {start_date} a {end_date}")

# Calcular el número de días
days = (end_date - start_date).days + 1

print(f"Total de días a generar: {days:,}")

# 2. Generar DataFrame base
dates_df = spark.range(0, days).select(
    F.lit(start_date).alias("base_date"),
    F.col("id").cast("int").alias("offset")
).select(
    F.expr("date_add(base_date, offset)").alias("date")
)

# 3. Extraer atributos para el análisis de tendencias
dim_date_df = dates_df.select(
    F.date_format(F.col("date"), "yyyyMMdd").cast("int").alias("date_key"),
    F.col("date"),
    F.year(F.col("date")).alias("year"),
    F.quarter(F.col("date")).alias("quarter"),
    F.month(F.col("date")).alias("month"),
    F.date_format(F.col("date"), "MMMM").alias("month_name")
)

# 4. CARGA INCREMENTAL CON MERGE
target_table = "dim_date"

if spark.catalog.tableExists(target_table):
    print(f"Tabla {target_table} existe — ejecutando MERGE incremental...")
    
    deltaTable = DeltaTable.forName(spark, target_table)
    
    # MERGE solo inserta fechas nuevas
    deltaTable.alias("target").merge(
        dim_date_df.alias("source"),
        "target.date = source.date"
    ).whenNotMatchedInsertAll().execute()
    
    print(f"✓ MERGE completado")
    
else:
    print(f"Primera carga — creando tabla {target_table}...")
    
    dim_date_df.write \
        .format("delta") \
        .mode("overwrite") \
        .saveAsTable(target_table)
    
    print(f"✓ Tabla creada exitosamente")

# Verificar resultado final
final_count = spark.table(target_table).count()
print(f"Total de fechas en dim_date: {final_count:,}")

StatementMeta(, c9f4b606-121f-449b-8c32-e9ec6f4c2020, 3, Finished, Available, Finished, False)

Generando dim_date
Rango detectado: 1970-01-01 a 2026-02-18
Total de días a generar: 20,503
Primera carga — creando tabla dim_date...
✓ Tabla creada exitosamente
Total de fechas en dim_date: 20,503


## FACT_NEWS

In [2]:
# 2. UNIFICACIÓN (Stacking)
fact_columns = [
    "id", "title", "url", "image_url", "summary", "published_at", "updated_at", "featured", 
    "organizations", "related_people", "places", "date_key", "source_id", "keyword_id"
]

# Columna adicional de content_type
def select_fact_columns(df, content_type):
    return df.select(
        *fact_columns,
        F.lit(content_type).alias("content_type")
    )

try:
    # Unión de las 3 fuentes
    fact_df = select_fact_columns(df_articles, "Article") \
        .union(select_fact_columns(df_blogs, "Blog")) \
        .union(select_fact_columns(df_reports, "Report"))

    # 3. Persistencia en el Lakehouse (Idempotente)
    target_table = "fact_news"
    
    if spark.catalog.tableExists(target_table):
        print(f"--> Actualizando {target_table} mediante MERGE...")
        dt = DeltaTable.forName(spark, target_table)
        
        # Merge por ID y tipo para evitar duplicados si un ID se repite entre fuentes
        dt.alias("target").merge(
            fact_df.alias("source"),
            "target.id = source.id AND target.content_type = source.content_type"
        ).whenNotMatchedInsertAll() \
         .execute()
    else:
        print(f"--> Creando {target_table} por primera vez...")
        fact_df.write.format("delta").mode("overwrite").saveAsTable(target_table)
        
    print("¡Tabla de hechos generada con éxito!")
    display(fact_df.limit(5))

except Exception as e:
    print(f"Error detectado: {e}")

StatementMeta(, c9f4b606-121f-449b-8c32-e9ec6f4c2020, 4, Finished, Available, Finished, False)

--> Creando fact_news por primera vez...
¡Tabla de hechos generada con éxito!


SynapseWidget(Synapse.DataFrame, 5442f2cf-2a03-47ef-b289-b9763bb0fc62)